<a href="https://colab.research.google.com/github/ShamaKolhar/ResearchProject/blob/main/dissertation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!pip install pydicom pypng


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 7.6 MB/s eta 0:00:00


In [17]:
!pip install -q pydicom pylibjpeg pylibjpeg-libjpeg pylibjpeg-openjpeg numpy


In [22]:
import os
from glob import glob
converted_files = glob("/content/drive/MyDrive/dissertation/pngimages/*.png")
print("✅ Total PNGs:", len(converted_files))


✅ Total PNGs: 341


In [27]:
import os
import pydicom
import pandas as pd

dcm_root = "/content/drive/MyDrive/dissertation/dcmimages/dcm"
records = []

for root, _, files in os.walk(dcm_root):
    for file in files:
        if file.endswith(".dcm"):
            try:
                dcm_path = os.path.join(root, file)
                ds = pydicom.dcmread(dcm_path, force=True)

                exam_id = getattr(ds, "PatientID", "unknown")
                view = getattr(ds, "ViewPosition", "unknown")
                laterality = getattr(ds, "ImageLaterality", "unknown")

                records.append({
                    "dicom_path": dcm_path,
                    "dicom_filename": file,
                    "exam_id": exam_id,
                    "view": view,
                    "laterality": laterality
                })
            except Exception as e:
                print(f"Failed on {file}: {e}")

df = pd.DataFrame(records)
df.to_csv("dicom_metadata.csv", index=False)
print("✅ Metadata saved to dicom_metadata.csv")


✅ Metadata saved to dicom_metadata.csv


In [31]:
import pandas as pd
import os

# Load your metadata CSV (from the previous step)
metadata_df = pd.read_csv("dicom_metadata.csv")

# Path where your PNG images were saved (change if needed)
png_folder = "/content/drive/MyDrive/dissertation/pngimages"

# Create list for the final records
records = []

# Match PNG files to DICOM metadata
for fname in os.listdir(png_folder):
    if not fname.lower().endswith(".png"):
        continue

    # Remove .png extension to match dicom filename
    dicom_id = fname.replace(".png", "")
    row = metadata_df[metadata_df["dicom_filename"].str.contains(dicom_id)]

    if not row.empty:
        row = row.iloc[0]
        records.append({
            "file_path": os.path.join(png_folder, fname),
            "exam_id": row["exam_id"],
            "view": row["view"],
            "laterality": row["laterality"]
        })
    else:
        print(f"Skipping unmatched PNG: {fname}")

# Save the final CSV
final_df = pd.DataFrame(records)
final_df.to_csv("mirai_metadata.csv", index=False)
print("✅ Saved mirai_metadata.csv with", len(final_df), "entries")


✅ Saved mirai_metadata.csv with 341 entries


In [37]:
!python /content/drive/MyDrive/dissertation/AsymMirai-master/AsymMirai-master/asymmetry_model/asymmetry_metrics.py

In [38]:
!python /content/drive/MyDrive/dissertation/AsymMirai-master/AsymMirai-master/asymmetry_model/mirai_localized_dif_head.py

In [45]:
import sys
sys.path.append("/content/drive/MyDrive/dissertation/AsymMirai-master/AsymMirai-master")


In [44]:
!ls /content
import os
for root, dirs, files in os.walk("/content"):
    for file in files:
        if file.endswith(".py"):
            print(os.path.join(root, file))


dicom_metadata.csv  drive  mirai_metadata.csv  sample_data
/content/drive/MyDrive/Research Project/AsymMirai-radiology-1.0/get_asymmetries.py
/content/drive/MyDrive/Research Project/AsymMirai-radiology-1.0/scripts/main.py
/content/drive/MyDrive/Research Project/AsymMirai-radiology-1.0/scripts/dispatcher.py
/content/drive/MyDrive/Research Project/AsymMirai-radiology-1.0/scripts/get_dataset_stats.py
/content/drive/MyDrive/Research Project/AsymMirai-radiology-1.0/tests/test_loader.py
/content/drive/MyDrive/Research Project/AsymMirai-radiology-1.0/tests/test_utils.py
/content/drive/MyDrive/Research Project/AsymMirai-radiology-1.0/tests/test_models.py
/content/drive/MyDrive/Research Project/AsymMirai-radiology-1.0/tests/test_transformers.py
/content/drive/MyDrive/Research Project/AsymMirai-radiology-1.0/tests/__init__.py
/content/drive/MyDrive/Research Project/AsymMirai-radiology-1.0/tests/test_asymmetry.py
/content/drive/MyDrive/Research Project/AsymMirai-radiology-1.0/onconet/__init__.py


In [86]:
%cd /content/drive/MyDrive/dissertation/AsymMirai-master/AsymMirai-master/asymmetry_model

!python train.py



/content/drive/MyDrive/dissertation/AsymMirai-master/AsymMirai-master/asymmetry_model
⚠️ Using default ResNet18 as backbone.
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
✅ Saved final MC Dropout predictions.
